In [2]:
import os
import math
import copy
import time
import random
import pprint
import tqdm
import platform
from datetime import datetime

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import torch
from torch import nn
from torch.nn import CrossEntropyLoss
import torch.nn.functional as F
from torchvision import utils
from torch.utils.data import Dataset, DataLoader, ConcatDataset
from torch.utils.tensorboard import SummaryWriter
from sklearn.model_selection import KFold
# from torchinfo import summary

import cv2
import nibabel as nib
import skimage.transform as skTrans
from numpy import logical_and as l_and, logical_not as l_not
from scipy.spatial.distance import directed_hausdorff

%matplotlib inline

%load_ext autoreload
%autoreload 2

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [4]:
# Set random seed for reproduciablity
torch.manual_seed(42)
random.seed(42)

In [5]:
class ScaleToFixed(object):

    def __init__(self, new_shape, interpolation=1, channels=4):
        self.shape= new_shape
        self.interpolation = interpolation
        self.channels = channels

    def __call__(self, image):
        # print('first shape', image.shape)
        if image is not None: # (some patients don't have segmentations)
            if self.channels == 1:
                short_shape = (self.shape[1], self.shape[2], self.shape[3])
                image = skTrans.resize(image, short_shape, order=self.interpolation, preserve_range=True)  #
                image = image.reshape(self.shape)
            else:
                image = skTrans.resize(image, self.shape, order=self.interpolation, preserve_range=True)  #

        # print('second shape', image.shape)
        # print()
        return image

class RandomFlip(object):
    """Randomly flips (horizontally as well as vertically) the given PIL.Image with a probability of 0.5
    """
    def __init__(self, prob_flip=0.5):
        self.prob_flip= prob_flip
    def __call__(self, image):

        if random.random() < self.prob_flip:
            flip_type = np.random.randint(0, 3) # flip across any 3D axis
            image = np.flip(image, flip_type)
        return image

class ZeroChannel(object):
    """Randomly sets channel to zero the given PIL.Image with a probability of 0.25
    """
    def __init__(self, prob_zero=0.25, channels=4):
        self.prob_zero= prob_zero
        self.channels = channels
    def __call__(self, image):

        if np.random.random() < self.prob_zero:
            channel_to_zero = np.random.randint(0, self.channels) # flip across any 3D axis
            zeros = np.zeros((image.shape[1], image.shape[2], image.shape[3]))
            image[channel_to_zero, :, :, :] = zeros
        return image

class ZeroSprinkle(object):
    def __init__(self, prob_zero=0.25, prob_true=0.5, channels=4):
        self.prob_zero=prob_zero
        self.prob_true=prob_true
        self.channels=channels
    def __call__(self, image):

        if self.prob_true:
            mask = np.random.rand(image.shape[0], image.shape[1], image.shape[2], image.shape[3])
            mask[mask < self.prob_zero] = 0
            mask[mask > 0] = 1
            image = image*mask

        return image


class MinMaxNormalize(object):
    """Min-Max normalization
    """
    def __call__(self, image):
        def norm(im):
            im = im.astype(np.float32)
            min_v = np.min(im)
            max_v = np.max(im)
            im = (im - min_v)/(max_v - min_v)
            return im
        image = norm(image)
        return image

class ToTensor(object):
    def __init__(self, scale=1):
        self.scale = scale

    def __call__(self, image):
        if image is not None:
            image = image.astype(np.float32)
            image = image.reshape((image.shape[0], int(image.shape[1]/self.scale), int(image.shape[2]/self.scale), int(image.shape[3]/self.scale)))
            image_tensor = torch.from_numpy(image)
            return image_tensor
        else:
            return image


class Compose(object):
    """
    Composes several transforms together.
    """

    def __init__(self, transforms):
        self.transforms = transforms

    def __call__(self, image):
        for i, t in enumerate(self.transforms):
            image = t(image)
        return image

In [6]:
def get_bb_3D(img, pad=0):
    '''
    This function returns a tumor 3D bounding box using a segmentation mask
    '''
    xs = np.nonzero(np.sum(np.sum(img, axis=1), axis=1))
    ys = np.nonzero(np.sum(np.sum(img, axis=0), axis=1))
    zs = np.nonzero(np.sum(np.sum(img, axis=0), axis=0))
    xmin, xmax = np.min(xs), np.max(xs)
    ymin, ymax = np.min(ys), np.max(ys)
    zmin, zmax = np.min(zs), np.max(zs)
    bbox = (xmin-pad, ymin-pad, zmin-pad, xmax+pad, ymax+pad, zmax+pad)
    return bbox

def min_max(img):
    '''
    Min-max normalization
    '''
    return (img - img.min()) / (img.max() - img.min())

def read_mri(mr_path_dict, pad=0):

    image_shape = nib.load(mr_path_dict['flair']).get_fdata().shape
    bb_seg = get_bb_3D(nib.load(mr_path_dict['flair']).get_fdata())
    (xmin, ymin, zmin, xmax, ymax, zmax) = bb_seg

    xmin = np.max([0, xmin-pad])
    ymin = np.max([0, ymin-pad])
    zmin = np.max([0, zmin-pad])

    xmax = np.min([image_shape[0]-1, xmax+pad])
    ymax = np.min([image_shape[1]-1, ymax+pad])
    zmax = np.min([image_shape[2]-1, zmax+pad])


    img_dict = {}
    for key in ['flair', 't1', 't1ce', 't2', 'seg']:
        img = nib.load(mr_path_dict[key])
        img_data = img.get_fdata()
        img_dict[key] = img_data[xmin:xmax, ymin:ymax, zmin:zmax]

    stacked_img = np.stack([min_max(img_dict['flair']), min_max(img_dict['t1']),min_max(img_dict['t1ce']),min_max(img_dict['t2'])], axis=0)
    return stacked_img, img_dict['seg']


In [7]:
def plot_(image, seg, predicted=False):
    #Overlay with Predicted
    img = image[slice, :, :, :].squeeze()
    img = utils.make_grid(img)
    img = img.detach().cpu().numpy()
    
    print(img.shape)
    
    # plot images
    plt.figure(figsize=(10, 8))
    img_list = [img[i].T for i in range(channels)] # 1 image per channel
    plt.imshow(np.hstack(img_list), cmap='Greys_r')
    
    ## plot segmentation mask ##
    seg_img = torch.tensor(pred[slice].squeeze())
    if not predicted:
        seg_img = torch.tensor(seg_img.numpy()[:, ::-1].copy()) #flip
    seg_img = utils.make_grid(seg_img).detach().cpu().numpy()
    
    print(np.unique(seg_img))

    plt.imshow(np.hstack([seg_img[0].T]), cmap='Greys_r', alpha=0.3)
    plt.show()
    

In [8]:
class GeneralDataset(Dataset):

    def __init__(self,
                metadata_df,
                root_dir,
                transform=None,
                seg_transform=None, ###
                dataformat=None, # indicates what shape (or content) should be returned (2D or 3D, etc.)
                returndims=None, # what size/shape 3D volumes should be returned as.
                output_shape=None,
                visualize=False,
                modality=None,
                pad=2,
                device='cpu'):
        """
        Args:
            metadata_df (string): Path to the csv file w/ patient IDs
            root_dir (string): Directory for MR images
            transform (callable, optional)
        """
        self.device=device
        self.metadata_df = metadata_df
        self.root_dir = root_dir
        self.transform = transform
        self.seg_transform = seg_transform
        self.returndims=returndims
        self.modality = modality
        self.pad = pad
        self.output_shape = output_shape

    def __len__(self):
        return len(self.metadata_df)

    def __getitem__(self, idx):
        #print(type(idx), idx)
        if torch.is_tensor(idx):
            idx = idx.tolist()

        BraTS20ID = self.metadata_df.iloc[idx].BraTS_2020_subject_ID

        # make dictonary of paths to MRI volumnes (modalities) and segmenation masks
        mr_path_dict = {}
        sequence_type = ['seg', 't1', 't1ce', 'flair', 't2']
        for seq in sequence_type:
            mr_path_dict[seq] = os.path.join(self.root_dir, BraTS20ID, BraTS20ID + '_'+seq+'.nii.gz')

        image, seg_image = read_mri(mr_path_dict=mr_path_dict, pad=self.pad)
        
        if seg_image is not None:
            if self.output_shape == 'wt':
                seg_image[np.nonzero(seg_image)] = 1 #only 0's and 1's for background and tumor
            else:
                seg_image[seg_image == 4] = 3 #0,1,2,3 for background and tumor regions

        if self.transform:
            image = self.transform(image)
        if self.seg_transform:
            seg_image = self.seg_transform(seg_image)
        else:
            print('no transform')
        # print(image.shape)
        return (image, seg_image), BraTS20ID

In [9]:
def read_dataframe(params):
    if platform.system() == 'Windows':
        naming = pd.read_csv(f"{params['image_dir']}\\name_mapping.csv")
    else:
        naming = pd.read_csv(f"{params['image_dir']}/name_mapping.csv")
    
    data_df = pd.DataFrame(naming['BraTS_2020_subject_ID'])

    # n_patients_to_train_with
    total_num_patients = len(data_df)

    assert sum(params['tr_va_te_split']) == 100
    tr_split = int((total_num_patients * params['tr_va_te_split'][0]) / 100)
    va_split = int((total_num_patients * params['tr_va_te_split'][1]) / 100)
    te_split = total_num_patients - (tr_split + va_split)

    print(f"Data is split into train: {tr_split}, validation: {va_split} and test: {te_split}")
                   
    train_df = data_df[: tr_split]
    valid_df = data_df[tr_split : (tr_split + va_split)]
    test_df = data_df[(tr_split + va_split) :]

    return train_df, valid_df, test_df

In [10]:
def retrieve_dataset(df, train=False):

    image_dir, channels, resize_shape, output_shape = params['image_dir'], \
                                                      params['channels'], \
                                                      params['resize_shape'], \
                                                      params['output_shape']

    # basic data augmentation
    prob_voxel_zero = 0 # 0.1
    prob_channel_zero = 0 # 0.5
    prob_true = 0 # 0.8
    randomflip = RandomFlip()

    # MRI transformations
    train_transformations = Compose([
        MinMaxNormalize(),
        ScaleToFixed((channels, resize_shape[0], resize_shape[1],
                      resize_shape[2]), interpolation=1, channels=channels),
        ZeroSprinkle(prob_zero=prob_voxel_zero, prob_true=prob_true),
        ZeroChannel(prob_zero=prob_channel_zero),
        randomflip,
        ToTensor()
        ])
    
    val_transformations = Compose([
            MinMaxNormalize(),
            ScaleToFixed((channels, resize_shape[0], resize_shape[1],
                          resize_shape[2]), interpolation=1, channels=channels),
            ToTensor(),
        ])

    # GT segmentation mask transformations
    seg_transformations = Compose([
        ScaleToFixed((1, resize_shape[0], resize_shape[1],
                      resize_shape[2]), interpolation=0, channels=1),
        randomflip,
        ToTensor(),
        ])
    
    if train:
        dataset = GeneralDataset(metadata_df=df, 
                                root_dir=image_dir,
                                transform=train_transformations,
                                seg_transform=seg_transformations,
                                returndims=resize_shape,
                                output_shape=output_shape)
    else:
        dataset = GeneralDataset(metadata_df=df, 
                                root_dir=image_dir,
                                transform=val_transformations,
                                seg_transform=seg_transformations,
                                returndims=resize_shape,
                                output_shape=output_shape)
    return dataset

In [11]:
def get_data(params):

    train_df, valid_df, test_df = read_dataframe(params)

    train_dataset = retrieve_dataset(train_df, train=True)
    
    valid_dataset = retrieve_dataset(valid_df)

    test_dataset = retrieve_dataset(test_df)
    

    train_loader = DataLoader(train_dataset, batch_size=params['train_batch_size'], shuffle=True)
    valid_loader = DataLoader(valid_dataset, batch_size=params['train_batch_size'], shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=params['test_batch_size'])

    return train_loader, valid_loader, test_loader

In [12]:

class ConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels, norm='b', num_groups=2, k_size=3, stride=1, padding=1):
        super(ConvBlock, self).__init__()
        self.conv3d = nn.Conv3d(in_channels=in_channels, out_channels=out_channels, kernel_size=k_size,
                                stride=stride, padding=padding)
        if norm == 'b':
            self.norm = nn.BatchNorm3d(num_features=out_channels)
        else:
            # use only one group if the given number of groups is greater than the number of channels
            if out_channels < num_groups:
                num_groups = 1
            assert out_channels % num_groups == 0, f'Expected out_channels{out_channels} in input to be divisible by num_groups{num_groups}'
            self.norm = nn.GroupNorm(num_groups=num_groups, num_channels=out_channels)

    def forward(self, x):
        x = self.norm(self.conv3d(x))
        x = F.elu(x) #!
        return x

class ConvTranspose(nn.Module):
    def __init__(self, in_channels, out_channels, k_size=3, stride=2, padding=1, output_padding=1):
        super(ConvTranspose, self).__init__()
        self.conv3d_transpose = nn.ConvTranspose3d(in_channels=in_channels,
                                                   out_channels=out_channels,
                                                   kernel_size=k_size,
                                                   stride=stride,
                                                   padding=padding,
                                                   output_padding=output_padding)

    def forward(self, x):
        return self.conv3d_transpose(x)


class EncoderBlock(nn.Module):
    def __init__(self, in_channels, init_features, norm='b', num_groups=2, model_depth=4, pool_size=2):
        super(EncoderBlock, self).__init__()
        self.root_feat_maps = init_features
        self.num_conv_blocks = 2
        self.module_dict = nn.ModuleDict()
        for depth in range(model_depth):
            feat_map_channels = 2 ** (depth + 1) * self.root_feat_maps
            for i in range(self.num_conv_blocks):
                self.conv_block = ConvBlock(in_channels=in_channels, out_channels=feat_map_channels, norm=norm, num_groups=num_groups)
                self.module_dict["conv_{}_{}".format(depth, i)] = self.conv_block
                in_channels, feat_map_channels = feat_map_channels, feat_map_channels * 2
            if depth == model_depth - 1:
                break
            else:
                self.pooling = nn.MaxPool3d(kernel_size=pool_size, stride=2, padding=0)
                self.module_dict["max_pooling_{}".format(depth)] = self.pooling

    def forward(self, x):
        down_sampling_features = []
        for k, op in self.module_dict.items():
            if k.startswith("conv"):
                x = op(x)
                #print(k, x.shape)
                if k.endswith("1"):
                    down_sampling_features.append(x)
            elif k.startswith("max_pooling"):
                x = op(x)
                #print(k, x.shape)

        return x, down_sampling_features


class DecoderBlock(nn.Module):
    def __init__(self, out_channels, init_features, norm, num_groups=2, model_depth=4):
        super(DecoderBlock, self).__init__()
        self.num_conv_blocks = 2
        self.num_feat_maps = init_features
        # user nn.ModuleDict() to store ops
        self.module_dict = nn.ModuleDict()

        for depth in range(model_depth - 2, -1, -1):
            # print(depth)
            feat_map_channels = 2 ** (depth + 1) * self.num_feat_maps
            # print(feat_map_channels * 4)
            self.deconv = ConvTranspose(in_channels=feat_map_channels * 4, out_channels=feat_map_channels * 4)
            self.module_dict["deconv_{}".format(depth)] = self.deconv
            for i in range(self.num_conv_blocks):
                if i == 0:
                    self.conv = ConvBlock(in_channels=feat_map_channels * 6, out_channels=feat_map_channels * 2, norm=norm, num_groups=num_groups)
                    self.module_dict["conv_{}_{}".format(depth, i)] = self.conv
                else:
                    self.conv = ConvBlock(in_channels=feat_map_channels * 2, out_channels=feat_map_channels * 2, norm=norm, num_groups=num_groups)
                    self.module_dict["conv_{}_{}".format(depth, i)] = self.conv
            if depth == 0:
                self.final_conv = ConvBlock(in_channels=feat_map_channels * 2, out_channels=out_channels, norm=norm, num_groups=num_groups)
                self.module_dict["final_conv"] = self.final_conv

    def forward(self, x, down_sampling_features):
        """
        :param x: inputs
        :param down_sampling_features: feature maps from encoder path
        :return: output
        """
        for k, op in self.module_dict.items():
            if k.startswith("deconv"):
                x = op(x)
                #print(k, x.shape)
                x = torch.cat((down_sampling_features[int(k[-1])], x), dim=1)
            elif k.startswith("conv"):
                x = op(x)
                #print(k, x.shape)
            else:
                x = op(x)
                #print(k, x.shape)
        return x


In [13]:
class UnetModel(nn.Module):

    def __init__(self, in_channels, out_channels, init_features, norm, num_groups=2, model_depth=4, final_activation="sigmoid"):
        super(UnetModel, self).__init__()
        self.encoder = EncoderBlock(in_channels=in_channels,
                                    init_features=init_features,
                                    norm=norm, num_groups=num_groups,
                                    model_depth=model_depth)
        self.decoder = DecoderBlock(out_channels=out_channels,
                                    init_features=init_features,
                                    norm=norm, num_groups=num_groups,
                                    model_depth=model_depth)
        if final_activation == "sigmoid":
            self.sigmoid = nn.Sigmoid()
        else:
            self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x, downsampling_features = self.encoder(x)
        x = self.decoder(x, downsampling_features)
        x = self.sigmoid(x)
        # print("Final output shape: ", x.shape)
        return x



In [14]:

class DiceLoss(nn.Module):
    def __init__(self, epsilon=1e-5):
        super(DiceLoss, self).__init__()
        # smooth factor
        self.epsilon = epsilon

    def forward(self, logits, targets):
        batch_size = targets.size(0)
        # log_prob = torch.sigmoid(logits)
        logits = logits.view(batch_size, -1).type(torch.FloatTensor)
        targets = targets.view(batch_size, -1).type(torch.FloatTensor)
        intersection = (logits * targets).sum(-1)
        dice_score = 2. * intersection / ((logits + targets).sum(-1) + self.epsilon)
        # dice_score = 1 - dice_score.sum() / batch_size
        dice_score = torch.mean(1. - dice_score)
        dice_score.requires_grad = True
        return dice_score

In [15]:
class EDiceLoss(nn.Module):
    """Dice loss tailored to Brats need.
    """

    def __init__(self, do_sigmoid=True):
        super(EDiceLoss, self).__init__()
        self.do_sigmoid = do_sigmoid
        self.labels = ["ET", "TC", "WT"]
        self.device = "cpu"

    def binary_dice(self, inputs, targets, label_index, metric_mode=False):
        smooth = 1.
        if self.do_sigmoid:
            inputs = torch.sigmoid(inputs)

        if metric_mode:
            inputs = inputs > 0.5
            if targets.sum() == 0:
                print(f"No {self.labels[label_index]} for this patient")
                if inputs.sum() == 0:
                    return torch.tensor(1., device=device)
                else:
                    return torch.tensor(0., device=device)
            # Threshold the pred
        intersection = EDiceLoss.compute_intersection(inputs, targets)
        if metric_mode:
            dice = (2 * intersection) / ((inputs.sum() + targets.sum()) * 1.0)
        else:
            dice = (2 * intersection + smooth) / (inputs.pow(2).sum() + targets.pow(2).sum() + smooth)
        if metric_mode:
            return dice
        return 1 - dice

    @staticmethod
    def compute_intersection(inputs, targets):
        intersection = torch.sum(inputs * targets)
        return intersection

    def forward(self, inputs, target):
        dice = 0
        for i in range(target.size(1)):
            dice = dice + self.binary_dice(inputs[:, i, ...], target[:, i, ...], i)
        final_dice = dice / target.size(1)
        final_dice.requires_grad = True
        return final_dice

    def metric(self, inputs, target):
        dices = []
        for j in range(target.size(0)):
            dice = []
            for i in range(target.size(1)):
                dice.append(self.binary_dice(inputs[j, i], target[j, i], i, True))
            dices.append(dice)
        return dices

In [16]:
def calculate_metrics(preds, targets, patient):
    """
    Parameters
    ----------
    preds:
        torch tensor of size 1*C*Z*Y*X, ours BS*Z*Y*X 
    targets:
        torch tensor of same shape
    patient :
        The patient ID
    """

    assert preds.shape == targets.shape, "Preds and targets do not have the same size"
    pp = pprint.PrettyPrinter(indent=4)
    
    preds, targets = preds.detach().cpu().numpy(), targets.detach().cpu().numpy()

    metrics_list = []

    metrics = dict(
        patient_id=patient,
    )
    # print(targets.shape, targets.dtype, targets)
    
    if np.sum(targets) == 0:
        print(f"{label} not present for {patient}")
    else:
        tp = np.sum(l_and(preds, targets))
        tn = np.sum(l_and(l_not(preds), l_not(targets)))
        fp = np.sum(l_and(preds, l_not(targets)))
        fn = np.sum(l_and(l_not(preds), targets))

        sens = tp / (tp + fn)
        spec = tn / (tn + fp)
        acc = (tn + tp) / (tn + tp + fn + fp)
        dice = 2 * tp / (2 * tp + fp + fn)

    metrics[DICE] = dice
    metrics[ACC] = acc
    metrics[SENS] = sens
    metrics[SPEC] = spec
    # pp.pprint(metrics)
    metrics_list.append(metrics)

    return acc, dice, metrics_list


DICE = "dice"
ACC = "acc"
SENS = "sens"
SPEC = "spec"
METRICS = [DICE, ACC, SENS, SPEC]


In [17]:
def calc_dice(preds, targets):
    return (2 * torch.sum(preds * targets)) / ((preds.sum() + preds.sum()) * 1.0)
    

In [18]:


def plot_metric(train, label, metric_name):
    # Plot losses
    plt.figure(figsize=(10,8))
    plt.semilogy(train, label=label)
    plt.xlabel('Epoch')
    plt.ylabel(metric_name)
    plt.legend()
    plt.title(f'Model {metric_name} Plot')
    plt.savefig(f'Model_{metric_name}_{label}_Plot.png')
    plt.show()
    plt.clf()

def plot_result(kfolds, num_epochs, fold_train_history, fold_valid_history):
    final_fold = {'train_loss':[],'valid_loss':[],'train_acc':[],'valid_acc':[]}

    for epoch in range(num_epochs):                                      
        final_fold['train_loss'].append(np.mean([fold_train_history[str(fold)]['train_loss'][epoch] for fold in range(kfolds)]))
        final_fold['train_acc'].append(np.mean([fold_train_history[str(fold)]['train_acc'][epoch]for fold in range(kfolds)]))

    plot_metric(final_fold['train_loss'], 'train', 'Loss')
    plot_metric(final_fold['train_acc'], 'validation', 'Accuracy')

    final_fold['valid_loss'].append([fold_valid_history[str(fold)]['valid_loss'] for fold in range(kfolds)])
    final_fold['valid_acc'].append([fold_valid_history[str(fold)]['valid_acc'] for fold in range(kfolds)])

    print(final_fold)

In [19]:
def load_checkpoint(path, params):

    test_model = UnetModel(params['pretrain_in_channels'],
                           params['pretrain_out_channels'],
                           params['init_features'], params['norm'],
                           params['num_groups'],
                           )

    test_optimizer = torch.optim.AdamW(test_model.parameters(),
                                       lr=params['learning_rate'],
                                       )
    
    use_cuda = torch.cuda.is_available()
    if use_cuda:
        checkpoint = torch.load(path)
    else:
        checkpoint = torch.load(path, map_location=torch.device('cpu'))

    test_model.load_state_dict(checkpoint['model_state_dict'])
    test_optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    epoch = checkpoint['epoch']
    loss = checkpoint['loss']

    return test_model, test_optimizer, epoch, loss

In [20]:
def save_model_folds(model, optimizer, fold, epoch, loss):
    # Saving the model
    save_path = f'model-fold-{fold}.pth'

    checkpoint = {'epoch': epoch,
                  'model_state_dict': model.state_dict(),
                  'optimizer_state_dict': optimizer.state_dict(),
                  'loss': loss,
                  }
    torch.save(checkpoint, save_path)


In [21]:
def save_model_nofolds(model, optimizer, epoch, loss, params):
    now = datetime.now()
    # dd/mm/YY H:M:S
    dt_string = now.strftime("%d_%H_%M")

    # Saving the model
    save_path = f"model_{params['output_shape']}_{params['run_name']}_{dt_string}.pth"

    checkpoint = {'epoch': epoch,
                  'model_state_dict': model.state_dict(),
                  'optimizer_state_dict': optimizer.state_dict(),
                  'loss': loss,
                  'params': params,
                  }
    torch.save(checkpoint, save_path)

In [22]:
def kFoldRunAll(criterion, dataset, params): 
                
    k_folds, num_epochs, train_batch_size = params['k_folds'],\
                                            params['no_epochs'],\
                                            params['train_batch_size']
    
    use_cuda, loss_name, in_channels, out_channels = params['use_cuda'], \
                                                     params['loss_name'],\
                                                     params['in_channels'],\
                                                     params['out_channels'],

    init_features, learning_rate, norm, num_groups = params['init_features'],\
                                                     params['learning_rate'],\
                                                     params['norm'], \
                                                     params['num_groups'],

    loss_function = criterion

    # Define the K-fold Cross Validator
    kfold = KFold(n_splits=k_folds, shuffle=True)

    fold_train_history = {}
    fold_valid_history = {}
    fold_train_and_valid_acc = {}
    fold_train_and_valid_loss = {}

    print('--------------------------------')
    # K-fold Cross Validation model evaluation
    for fold, (train_ids, test_ids) in enumerate(kfold.split(dataset)):
        print(f'FOLD {fold}')
        print('--------------------------------')
        # Sample elements randomly from a given list of ids, no replacement.
        train_subsampler = torch.utils.data.SubsetRandomSampler(train_ids)
        valid_subsampler = torch.utils.data.SubsetRandomSampler(test_ids)

        # Define data loaders for training and testing data in this fold
        dataloader_train = DataLoader(dataset, batch_size=train_batch_size, sampler=train_subsampler, num_workers=0)
        dataloader_valid = DataLoader(dataset, batch_size=train_batch_size, sampler=valid_subsampler, num_workers=0)

        # Initialize optimizer and Model
        model = UnetModel(in_channels=in_channels, out_channels=out_channels,
                          init_features=init_features, norm=norm, num_groups=num_groups)
        if use_cuda:
            model = model.cuda()
        #print(model)
        optimizer = torch.optim.Adam(params=model.parameters(), lr=learning_rate)

        # Run the training, testing and saving loop for defined number of epochs
        start_time = time.time()

        t_loss, t_acc, t_history, v_loss, v_acc, v_history = train_test_folds(model, loss_function, optimizer,
                                                                               dataloader_train, dataloader_valid,
                                                                               fold, num_epochs, use_cuda,
                                                                               loss_name)

        end_time = time.time()
        print(f"Epoch Time: {end_time - start_time}")

        #Saving loss results 
        fold_train_and_valid_loss[str(fold)] = [t_loss, v_loss]
        fold_train_and_valid_acc[str(fold)] = [t_acc, v_acc]
        fold_train_history[str(fold)] = t_history
        fold_valid_history[str(fold)] = v_history

        # Print accuracy
        print(f'Accuracy for fold {fold}: {v_acc}')
        print(f'Loss for fold {fold}: {v_loss}')
        print('--------------------------------')  

    return fold_train_history, fold_valid_history, fold_train_and_valid_loss, fold_train_and_valid_acc
  


In [23]:
def train_test_folds(model, loss_function, optimizer, dataloader_train, dataloader_valid, fold, num_epochs, use_cuda, loss_name):
    train_history = {'train_loss': [], 'train_acc':[], 'train_dice':[]}
    valid_history = {'valid_loss': [], 'valid_acc':[], 'valid_dice':[]}
    best = math.inf

    edice = EDiceLoss()
    if use_cuda:
        edice = edice.cuda()
    metric = edice.metric
        
    for epoch in range(num_epochs):
        print(f'Starting Train epoch: {epoch+1}')

        train_loss = 0.0
        train_acc, train_dice = 0, 0
        model.train()

        for i, data in enumerate(tqdm.tqdm(dataloader_train)):
            (inputs, targets), ID = data
            if use_cuda:
                inputs, targets = inputs.cuda(), targets.cuda()

            optimizer.zero_grad()
            outputs = model(inputs)
            #print(inputs.shape, outputs.shape, targets.squeeze(1).long().shape)

            if loss_name == 'dice':
                class_outputs = outputs.argmax(dim=1)
                loss = loss_function(class_outputs, targets.squeeze(1).long())
                # print(np.unique(class_outputs.detach().numpy()), class_outputs.shape, targets.squeeze(1).shape)
            else:
                loss = loss_function(outputs, targets.squeeze(1).long())

            train_loss += loss.item() * outputs.size(0) #multiplying by batchsize
            
            rtrain_dice1 = calc_dice(outputs.argmax(dim=1), targets.squeeze(1))
            rtrain_acc, rtrain_dice2, _ = calculate_metrics(outputs.argmax(dim=1), targets.squeeze(1), ID)
            print(f'Train Dice 1: {rtrain_dice1}, 2: {rtrain_dice2} \t Acc: {rtrain_acc}')
            train_acc += rtrain_acc
            train_dice += rtrain_dice1
            
            print(f'Train Loss :{loss.item()}')

            loss.backward()
            optimizer.step()
           
        train_history['train_loss'].append(train_loss / len(dataloader_train.sampler))
        train_history['train_acc'].append(train_acc / len(dataloader_train.sampler))
        train_history['train_dice'].append(train_dice / len(dataloader_train.sampler))

        print(f"Train Epoch loss: {train_history['train_loss'][-1]}, \t ACC/DICE :{train_history['train_acc'][-1]}/{train_history['train_dice'][-1]} ")

    valid_loss = 0.0
    valid_acc, valid_dice = 0, 0
           
    model.eval()
    #! maybe change later to validate after some epochs
    with torch.no_grad():
        # Iterate over the test data and generate predictions
        for i, data in enumerate(tqdm.tqdm(dataloader_valid)):
            (inputs, targets), ID = data
            if use_cuda:
                inputs, targets = inputs.cuda(), targets.cuda() 
            outputs = model(inputs)

            if params['loss_name'] == 'dice':
                class_outputs = outputs.argmax(dim=1)
                loss = loss_function(class_outputs, targets.squeeze(1).long())
                # print(np.unique(class_outputs.detach().numpy()), class_outputs.shape, targets.squeeze(1).shape)
            else:
                loss = loss_function(outputs, targets.squeeze(1).long())

            # print('Valid Loss:', loss.item())
            valid_loss += loss.item() * inputs.size(0)
            
            rvalid_dice1 = calc_dice(outputs.argmax(dim=1), targets.squeeze(1))

            rvalid_acc, rvalid_dice2, _ = calculate_metrics(outputs.argmax(dim=1), targets.squeeze(1), ID)
            # print(f'Val Dice 1: {rvalid_dice1}, 2: {rvalid_dice2}')
            valid_acc += rvalid_acc
            valid_dice += rvalid_dice1
            
        # Print accuracy
        print(f'Val Dice : {valid_dice}, len {len(dataloader_valid.sampler)}')
        valid_loss /= len(dataloader_valid.sampler) 
        valid_acc = valid_acc / len(dataloader_valid.sampler)
        valid_dice = valid_dice / len(dataloader_valid.sampler)
        #print(f" Fold Accuracy: {valid_acc}")

    valid_history['valid_loss'].append(valid_loss)
    valid_history['valid_acc'].append(valid_acc)
    valid_history['valid_dice'].append(valid_dice)

    print(f"Val Epoch loss: {valid_history['valid_loss'][-1]} \t acc/dice:/ {valid_history['valid_acc'][-1]}/ {valid_history['valid_dice'][-1]}")

    # saving best model for this fold
    if valid_loss < best:
        best = valid_loss
        save_model_folds(model, optimizer, fold, epoch, loss)
    
    
    return train_history['train_loss'][-1], train_history['train_acc'][-1], train_history, valid_history['valid_loss'][-1], valid_history['valid_acc'][-1], valid_history


In [24]:
def train_test_nofolds(model, loss_function, optimizer, dataloader_train, dataloader_valid, params):
    train_history = {'train_loss': [], 'train_acc':[], 'train_dice':[]}
    valid_history = {'valid_loss': [], 'valid_acc':[], 'valid_dice':[]}
    best = math.inf

    edice = EDiceLoss()
    if params['use_cuda']:
        edice = edice.cuda()
    metric = edice.metric

    #Automatic mixed precision addition
    #scaler = torch.cuda.amp.GradScaler()
        
    for epoch in range(params['no_epochs']):
        print(f'Starting Train epoch: {epoch+1}')

        train_loss = 0.0
        train_acc, train_dice = 0, 0
        model.train()

        for i, data in enumerate(tqdm.tqdm(dataloader_train)):
            (inputs, targets), ID = data
            if params['use_cuda']:
                inputs, targets = inputs.cuda(), targets.cuda()

            optimizer.zero_grad()
            #with torch.cuda.amp.autocast(enabled=params['autocast']):
            outputs = model(inputs)
            #print(inputs.shape, outputs.shape, targets.squeeze(1).long().shape)

            if params['loss_name'] == 'dice':
                class_outputs = outputs.argmax(dim=1)
                loss = loss_function(class_outputs, targets.squeeze(1).long())
                # print(np.unique(class_outputs.detach().numpy()), class_outputs.shape, targets.squeeze(1).shape)
            else:
                loss = loss_function(outputs, targets.squeeze(1).long())

            train_loss += loss.item() * outputs.size(0) # multiplying by batchsize
            
            rtrain_dice1 = calc_dice(outputs.argmax(dim=1), targets.squeeze(1))
            rtrain_acc, rtrain_dice2, _ = calculate_metrics(outputs.argmax(dim=1), targets.squeeze(1), ID)

            if i % 10 == 0:
                print(f'Train Dice 1: {rtrain_dice1}, 2: {rtrain_dice2} \t Acc: {rtrain_acc}')

            train_acc += rtrain_acc
            train_dice += rtrain_dice1
            if i % 10 == 0:
                print(f'Train Loss :{loss.item()}')
            
            #scaler.scale(loss).backward()
            #scaler.step(optimizer)
            #scaler.update()
            loss.backward()
            optimizer.step()
           
        train_history['train_loss'].append(train_loss / len(dataloader_train.sampler))
        train_history['train_acc'].append(train_acc / len(dataloader_train.sampler))
        train_history['train_dice'].append(train_dice / len(dataloader_train.sampler))

        print(f"Train Epoch loss: {train_history['train_loss'][-1]}, \t ACC/DICE :{train_history['train_acc'][-1]}/{train_history['train_dice'][-1]} ")
          
        if epoch % (.1 * params['no_epochs']) == 0:
            valid_loss = 0.0
            valid_acc, valid_dice = 0, 0
            model.eval()

            with torch.no_grad():
                # Iterate over the test data and generate predictions
                for i, data in enumerate(tqdm.tqdm(dataloader_valid)):
                    (inputs, targets), ID = data
                    if params['use_cuda']:
                        inputs, targets = inputs.cuda(), targets.cuda() 
                      
                    #with torch.cuda.amp.autocast(enabled=params['autocast']):
                    outputs = model(inputs)

                    if params['loss_name'] == 'dice':
                        class_outputs = outputs.argmax(dim=1)
                        loss = loss_function(class_outputs, targets.squeeze(1).long())
                        # print(np.unique(class_outputs.detach().numpy()), class_outputs.shape, targets.squeeze(1).shape)
                    else:
                        loss = loss_function(outputs, targets.squeeze(1).long())

                    # print('Valid Loss:', loss.item())
                    valid_loss += loss.item() * inputs.size(0)
                    
                    rvalid_dice1 = calc_dice(outputs.argmax(dim=1), targets.squeeze(1))

                    rvalid_acc, rvalid_dice2, _ = calculate_metrics(outputs.argmax(dim=1), targets.squeeze(1), ID)
                    # print(f'Val Dice 1: {rvalid_dice1}, 2: {rvalid_dice2}')
                    valid_acc += rvalid_acc
                    valid_dice += rvalid_dice1
                    
                # Print accuracy
                print(f'Val Dice : {valid_dice}, len {len(dataloader_valid.sampler)}')
                valid_loss /= len(dataloader_valid.sampler) 
                valid_acc = valid_acc / len(dataloader_valid.sampler)
                valid_dice = valid_dice / len(dataloader_valid.sampler)

            valid_history['valid_loss'].append(valid_loss)
            valid_history['valid_acc'].append(valid_acc)
            valid_history['valid_dice'].append(valid_dice)

            print(f"Val Epoch loss: {valid_history['valid_loss'][-1]} \t acc/dice:/ {valid_history['valid_acc'][-1]}/ {valid_history['valid_dice'][-1]}")

            # saving best model for this fold
            if valid_loss < best:
                best = valid_loss
                save_model_nofolds(model, optimizer, epoch, loss, params)
    
    
    return train_history['train_loss'][-1], train_history['train_acc'][-1], train_history, valid_history['valid_loss'][-1], valid_history['valid_acc'][-1], valid_history


In [25]:
def kfolds(params):
    
    if params['loss_name'] == 'ce':
        criterion = CrossEntropyLoss()
    elif params['loss_name'] == 'wce':
        wisdom_weights = [1, 355.36116969, 74.37872817, 254.58104099]
        nick_weights = [ 1.        ,  8.9263424 ,  7.79622053, 31.17438108]
        criterion = CrossEntropyLoss(weight=torch.Tensor(nick_weights))
    else:
        criterion = EDiceLoss().cuda()
      
    if params['use_cuda']:
        criterion.cuda()
    
    train_df, valid_df, _ = read_dataframe(params)

    train_valid_df = pd.concat([train_df, valid_df])

    train_valid_dataset = retrieve_dataset(train_valid_df)

    t_history, v_history, tv_loss, tv_acc = kFoldRunAll(criterion, 
                                                        train_valid_dataset,
                                                        params)
    
    plot_result(k_folds, no_epochs, t_history, v_history)

    return t_history, v_history, tv_loss, tv_acc

In [26]:
def transfer(path, params):
    
    if params['loss_name'] == 'ce':
        criterion = CrossEntropyLoss()
    elif params['loss_name'] == 'wce':
        criterion = CrossEntropyLoss(weight=torch.Tensor([1, 355.36116969, 74.37872817, 254.58104099]))
    else:
        criterion = EDiceLoss().cuda()
    
    model, optimizer, _, __ = load_checkpoint(path, params)

    # Transfer by changing(replacing) only last layer and finetuning to outdim=4
    model.decoder.final_conv = ConvBlock(in_channels=params['pretrain_in_final_conv'], 
                                          out_channels=params['out_channels'],
                                          norm=params['norm'],
                                          num_groups=params['num_groups'])

    model.decoder.module_dict.final_conv = ConvBlock(in_channels=params['pretrain_in_final_conv'], 
                                          out_channels=params['out_channels'],
                                          norm=params['norm'],
                                          num_groups=params['num_groups'])

    if params['use_cuda']:
        model.cuda()
        criterion.cuda()
    
    dataloader_train, dataloader_valid, _ = get_data(params)
    
    t_loss, t_acc, t_history, v_loss, v_acc, v_history = train_test_nofolds(
                                                model, criterion, optimizer,
                                                dataloader_train, dataloader_valid,
                                                params
                                                )

    # plot_result(k_folds, params['no_epochs'], t_history, v_history)

    return t_loss, t_acc, t_history, v_loss, v_acc, v_history

In [27]:
def learn_from_scratch(params):
    
    if params['loss_name'] == 'ce':
        criterion = CrossEntropyLoss()
    elif params['loss_name'] == 'wce':
        criterion = CrossEntropyLoss(weight=torch.Tensor([1, 355.36116969, 74.37872817, 254.58104099]))
    else:
        criterion = EDiceLoss()
    


    model = UnetModel(params['in_channels'],
                      params['out_channels'],
                      params['init_features'],
                      params['norm'],
                      params['num_groups'],
                      )

    optimizer = torch.optim.AdamW(model.parameters(),
                                  lr=params['learning_rate'],
                                  )

    if params['use_cuda']:
        model.cuda()
        criterion.cuda()
    
    dataloader_train, dataloader_valid, _ = get_data(params)
    
    t_loss, t_acc, t_history, v_loss, v_acc, v_history = train_test_nofolds(
                                                model, criterion, optimizer,
                                                dataloader_train, dataloader_valid,
                                                params
                                                )

    # plot_result(k_folds, params['no_epochs'], t_history, v_history)

    return t_loss, t_acc, t_history, v_loss, v_acc, v_history

In [28]:
def get_params():

    if platform.system() == 'Windows':
        image_dir = r"C:\Users\wisdomik\Documents\project\MICCAI_BraTS2020_TrainingData"
    else:
        image_dir = '../../data/data/mri/MICCAI_BraTS2020_TrainingData/'
    
    use_cuda = torch.cuda.is_available()

    params = {'run_name': 'run_1',
              'in_channels': 4,
              'out_channels': 4,
              'no_epochs': 20,
              'k_folds': 5,
              'learning_rate': 5e-4, # 1e-4,
              'loss_name': 'wce',
              'output_shape': 'all',
              'tr_va_te_split': [75, 25, 0],
              'pretrain_in_channels': 4,
              'pretrain_out_channels': 2,
              'pretrain_in_final_conv': 16,
              'init_features': 8,
              'train_batch_size': 1,
              'autocast': False, # not in use
              'test_batch_size': 1,
              'norm': 'g',
              'num_groups': 4,
              'channels': 4,
              'resize_shape': (144, 144, 144), # 128
              'image_dir': image_dir,
              'use_cuda': use_cuda
              }
              
    return params



In [30]:
# set params then run this

#load in trained model for evaluation
# fold_to_check = 9
# PATH = f'drive/MyDrive/Colab Notebooks/model-fold-{fold_to_check}.pth'

params = get_params()
params

{'run_name': 'run_1',
 'in_channels': 4,
 'out_channels': 4,
 'no_epochs': 20,
 'k_folds': 5,
 'learning_rate': 0.0005,
 'loss_name': 'wce',
 'output_shape': 'all',
 'tr_va_te_split': [75, 25, 0],
 'pretrain_in_channels': 4,
 'pretrain_out_channels': 2,
 'pretrain_in_final_conv': 16,
 'init_features': 8,
 'train_batch_size': 1,
 'autocast': False,
 'test_batch_size': 1,
 'norm': 'g',
 'num_groups': 4,
 'channels': 4,
 'resize_shape': (144, 144, 144),
 'image_dir': '../../data/data/mri/MICCAI_BraTS2020_TrainingData/',
 'use_cuda': True}

In [31]:
# Run learn from scratch (UNCOMMENT TO RUN)

t_loss, t_acc, t_history, v_loss, v_acc, v_history = learn_from_scratch(params)

  0%|          | 0/276 [00:00<?, ?it/s]

Data is split into train: 276, validation: 92 and test: 1
Starting Train epoch: 1
Train Dice 1: 0.025612566620111465, 2: 0.04134624607639819 	 Acc: 0.22347474065500686
Train Loss :1.4356637001037598


  4%|▍         | 11/276 [00:49<19:16,  4.37s/it]

Train Dice 1: 0.04484144598245621, 2: 0.04583972751076452 	 Acc: 0.32659250685871055
Train Loss :1.5656408071517944


  8%|▊         | 21/276 [01:35<21:54,  5.15s/it]

...

Train Dice 1: 0.10746656358242035, 2: 0.1062131579314093 	 Acc: 0.6563290359224966
Train Loss :1.230942964553833


 98%|█████████▊| 271/276 [17:30<00:19,  3.89s/it]

Train Dice 1: 0.3514087498188019, 2: 0.27758007800503653 	 Acc: 0.7479055480538409
Train Loss :1.3067092895507812


100%|██████████| 276/276 [17:49<00:00,  3.88s/it]


Train Epoch loss: 1.2073662710794504, 	 ACC/DICE :0.6854010706135001/0.160820871591568 


In [32]:
# torch.cuda.empty_cache()